#  数据清洗：表格数据缺失值与异常值的处理

**为什么需要做数据异常处理**：现实工作中，因为在数据记录和数据存储环节偶尔会出现问题，比如互联网公司后端的行为日志记录系统时不时就会出现问题，导致部分数据的丢失。所以数据分析师拿到的原始数据中会存在很多字段或者记录是丢失的。为了不让这些缺失的数据影响数据分析的结果，在分析之前往往就需要进行数据清洗，对这些缺失的数据进行预处理。

## 1 什么是缺失值

当我们从 CSV 文件或者其他数据源加载到 DataFrame 中时，往往会遇到某些单元格的数据是缺失的。当我们打印出 DataFrame 时，缺失的部分会显示为 NaN， 或者 None，或者 NaT（取决于单元格的数据类型），这样的值我们就称之为缺失值。

参考下面 DataFrame：

In [23]:
import pandas as pd

# DataFrame 的列名
index_arr = ["听力", "阅读", "写作", "口试"]

scores = [
    [20.26, 71.58, 27.06, 97.51],
    [40.61, 72.32, 56.54, 5.45],
    [72.44, 68.89, 6.65, 75.54]
]

# 从 scores 列表中创建 DataFrame
# index 参数代表行索引
# columns 参数代表列索引
df_scores = pd.DataFrame(scores, index=["小亮", "小明", "小E"], columns=index_arr)

# 生成小李的 Series，没有的成绩用 None 取代
ser_xl = pd.Series([30.04, None, None, None], index=index_arr, name="小李")
# 将小李的 Series 添加到 df_scores 中
df_scores = df_scores.append(ser_xl)

# 生成小王的 Series，没有的成绩用 None 取代
ser_xw = pd.Series([None, 91.00, 72.34, None], index=index_arr, name="小王")
# 将小王的 Series 添加到 df_scores 中
df_scores = df_scores.append(ser_xw)

# 查看 df_scores
df_scores

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5700\2324112345.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append(ser_xl)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_5700\2324112345.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append(ser_xw)


,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN
小王,NaN,91.00,72.34,NaN


可以看到，小李的阅读、写作和口试显示了 NaN，代表数字类型的缺失值（小王也类似）。时间类型的缺失值一般显示为 NaT，而字符串类型的则显示为 None。在实际项目中，缺失值可以说一直存在于原始的数据源中。如果我们在数据分析时不把它处理掉，很可能会**得到错误的结果**。比如如果要计算写作科目的平均分，小李的 NaN 到底是当作 0，还是当作平均数，还是干脆就不把小李纳入计算，都需要**根据情况进行决策，来最大化降低缺失值对于分析结果的影响**。

## 2 查询缺失值

处理缺失值，首先第一步是查询缺失值是否存在，以及数量情况如何。

In [4]:
# 1：按单元格查看缺失值情况
df_scores.isna()

,听力,阅读,写作,口试
小亮,False,False,False,False
小明,False,False,False,False
小E,False,False,False,False
小李,False,True,True,True
小王,True,False,False,True


In [5]:
# 2：按列查看缺失值
df_scores.isna().sum()

听力    1
阅读    1
写作    1
口试    2
dtype: int64

In [6]:
# 3：按行查看缺失值
df_scores.isna().sum(1)

小亮    0
小明    0
小E    0
小李    3
小王    2
dtype: int64

In [7]:
# 4：过滤出有缺失值的列
# 行索引部分，取所有的行
# 列索引部分，取所有包含缺失值的列
# any 函数类似 sum 函数，但 any 函数做的是布尔聚合，当列有一个或以上的 True 时，结果就是 True， 否则为 False
df_scores.loc[:, df_scores.isna().any()]

# 因为目前我们的 DataFrame 每一列都至少包含一个缺失值，所以过滤列之后输出了所有记录。

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN
小王,NaN,91.00,72.34,NaN


In [8]:
# 5：过滤出有缺失值的行
# 行索引部分，通过 any(1) 来聚合行维度的结果
# 列索引部分，取所有的列
df_scores.loc[df_scores.isna().any(1), :]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_19336\1022536686.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df_scores.loc[df_scores.isna().any(1),:]


,听力,阅读,写作,口试
小李,30.04,NaN,NaN,NaN
小王,NaN,91.0,72.34,NaN


In [9]:
# 6：缺失值的总个数
# 获取整个 DataFrame 一共包含多少个缺失值。
df_scores.isna().sum().sum()

5

## 3 处理缺失值

常见的缺失值处理方法有以下三种。

1. 缺失值删除
2. 缺失值替换
3. 缺失值插值

另外，还需要处理重复值。

### 缺失值删除

In [3]:
# 删除所有缺失值所在的行
df_scores.dropna()

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54


In [4]:
# 删除所有缺失值所在的列，这里因为我们的 DataFrame 每一列都至少有一个缺失值，所以删除后 DataFrame 只剩下行索引。
df_scores.dropna(axis=1)

""
小亮
小明
小E
小李
小王


In [5]:
# 删除少于 X 个正常值的行【小李的正常值只有 1 个，所以被删除。而小王的正常值有两个，所以被保留。】
df_scores.dropna(thresh=2)

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小王,NaN,91.00,72.34,NaN


In [6]:
# 参考某几列作为删除依据
# 我们的数据表中不同的列权重（重要性）是不一样的。比如这次职工英语考试，最关键的是听力，所以我们希望只看听力这一列，如果听力是缺失值，则删除，其他列有缺失值则不删除。可以通过 subset 参数实现。
df_scores.dropna(subset=["听力"])

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN


另外，需要注意一点的是，dropna 方法默认不会改变调用它的 DataFrame，而是会将删除缺失值后的 DataFrame 作为函数的返回值返回。所以上面的代码并没有实际修改到 df_scores。如果需要实际修改 df_scores ，则需要做一次赋值，比如： `df_scores = df_scores.dropna()`。

### 缺失值替换

除了删除之外，另一个主流的缺失值处理方式就是替换。简单来说就是将缺失值的部分替换为一个固定的值，来减少缺失值带来的对于分析结果的不确定性。当数据量大且缺失值的数量也不小的时候，使用填充策略相比删除策略能显著提升分析结果的准确性。

缺失值替换有以下几种策略：

In [7]:
# 全表固定值替换【用 33.0 这个数字来替换掉全部的缺失值。】
df_scores.fillna(33.0)

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,33.00,33.00,33.00
小王,33.00,91.00,72.34,33.00


In [10]:
# 按列固定值替换
df_scores_test = df_scores.copy(deep=True)
df_scores_test["听力"] = df_scores_test["听力"].fillna(60.0)
df_scores_test

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN
小王,60.00,91.00,72.34,NaN


In [12]:
# 按行固定值替换
df_scores_test.loc["小李", :] = df_scores_test.loc["小李", :].fillna("50.0")
df_scores_test

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,50.0,50.0,50.0
小王,60.00,91.0,72.34,NaN


In [13]:
# 最近有效值替换：什么叫最近有效值呢？就是在列的维度，当某一个单元格的数据是缺失值时，在该列往上搜索，碰到第一个有效值（非缺失值），就是最近有效值。
df_scores.fillna(method="ffill")

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,68.89,6.65,75.54
小王,30.04,91.00,72.34,75.54


In [14]:
# 当我们设置 method="bfill" 的时候，pandas 就会用缺失值对应列，往下搜索的第一个有效值来填充
df_scores.fillna(method="bfill")

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,91.00,72.34,NaN
小王,NaN,91.00,72.34,NaN


### 缺失值插值

在有的场景下，只使用最近有效值依然不能很好地满足分析的诉求。比如一些时间序列分析的场景，缺失值可能和前面或者后面的数据都有一定的关系。

如果可以结合缺失值前后的有效值的信息来推测缺失值，那准确性相比直接用最近有效值要高很多。pandas 提供了插值方法来实现这一目的。

插值简单来说就是通过已经有的点来拟合出一个函数关系（f），然后根据缺失值的位置（x）来去拟合出来的函数中拿到对应的 f(x) 值，然后用这个值去替换掉缺失值。这样我们认为这个 f(x) 是最有可能贴近真实的值的。

插值的方法有很多，最简单的有线性插值、临近点插值、立方插值等。这里以简单的线性插值为例来介绍 pandas 插值的用法。

In [16]:
ser_test = pd.Series([100, 3, None, None, 9])
ser_test

0    100.0
1      3.0
2      NaN
3      NaN
4      9.0
dtype: float64

In [17]:
# 目前 ser_test 中有两个缺失值，想要通过线性插值来计算出这两个缺失值的话，我们可以拿到缺失值前后的两个数据点(1,3.0), (4, 9.0)，根据两点直线方程有：
ser_test.interpolate()

0    100.0
1      3.0
2      5.0
3      7.0
4      9.0
dtype: float64

In [18]:
df_scores.interpolate()

,听力,阅读,写作,口试
小亮,20.26,71.580,27.060,97.51
小明,40.61,72.320,56.540,5.45
小E,72.44,68.890,6.650,75.54
小李,30.04,79.945,39.495,75.54
小王,30.04,91.000,72.340,75.54


可以看到小王的阅读和写作两个缺失值成功替换为了线性插值的版本，而其他缺失部分却仍然是用的最近有效值，这是为何呢？其实很简单，线性插值需要缺失值前后有效值的信息来拟合方程，而红框部分都缺少后面的有效值，所以无法拟合。当线性插值无法拟合的时候，会默认采用最近有效值来填充。

### 处理重复值

除了常见的缺失值之外，实际项目中还经常遇到的异常数据问题就是重复值。企业的数据日志记录系统出现问题时，有时候会导致丢失数据，这就产生了缺失值的问题。有的时候会重复写入数据，这也产生了重复值的问题。

重复值指的是 DataFrame 中的两行全部或部分一样。

In [19]:
# 生成一条一模一样的小王的记录
ser_xw = pd.Series([None, 91.00, 72.34, None], index=index_arr, name="小王")

# 将新增加的两 Series 添加到 df_scores 中
df_scores = df_scores.append(ser_xw)

# 查看 df_scores
df_scores

C:\Users\lenovo\AppData\Local\Temp\ipykernel_5700\3685348130.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_scores = df_scores.append(ser_xw)


,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN
小王,NaN,91.00,72.34,NaN
小王,NaN,91.00,72.34,NaN


In [25]:
# 只需要调用 pandas 提供的 drop_duplicates 方法即可删除重复值。
df_scores = df_scores.drop_duplicates()
df_scores

,听力,阅读,写作,口试
小亮,20.26,71.58,27.06,97.51
小明,40.61,72.32,56.54,5.45
小E,72.44,68.89,6.65,75.54
小李,30.04,NaN,NaN,NaN
小王,NaN,91.00,72.34,NaN


## 4 内容总结

缺失值的概念：DataFrame 中缺少的部分数据，数字的显示为 NaN，字符串显示为 None，时间类型则显示为 NaT。

查看缺失值：

```
按单元格查看：df.isna()
按列查看：df.isna().sum()
按行查看：df.isna().sum(1)
有缺失值的列：df.loc[:, df.isna().any()]
有缺失值的行：df_scores.loc[df_scores.isna().any(1),:]
缺失值总个数：df.isna().sum().sum()
```

处理缺失值：

```
删除缺失值：df.dropna()
缺失值替换：df.fillna()
缺失值插值：df.interpolate()
```

处理重复值：

```
查看重复行：df.duplicated()
删除重复行：df.drop_duplicates()
```